# Applying Classification Modeling
The goal of this week's assessment is to find the model which best predicts whether or not a person will default on their bank loan. In doing so, we want to utilize all of the different tools we have learned over the course: data cleaning, EDA, feature engineering/transformation, feature selection, hyperparameter tuning, and model evaluation. 


#### Data Set Information:

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients. Because the real probability of default is unknown, this study presented the novel Sorting Smoothing Method to estimate the real probability of default. With the real probability of default as the response variable (Y), and the predictive probability of default as the independent variable (X), the simple linear regression result (Y = A + BX) shows that the forecasting model produced by artificial neural network has the highest coefficient of determination; its regression intercept (A) is close to zero, and regression coefficient (B) to one. Therefore, among the six data mining techniques, artificial neural network is the only one that can accurately estimate the real probability of default. 

- NT is the abbreviation for New Taiwain. 


#### Attribute Information:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables: 
- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
- X2: Gender (1 = male; 2 = female). 
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- X4: Marital status (1 = married; 2 = single; 3 = others). 
- X5: Age (year). 
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: 
    - X6 = the repayment status in September, 2005; 
    - X7 = the repayment status in August, 2005; . . .;
    - etc...
    - X11 = the repayment status in April, 2005. 
    - The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- X12-X17: Amount of bill statement (NT dollar). 
    - X12 = amount of bill statement in September, 2005;
    - etc...
    - X13 = amount of bill statement in August, 2005; . . .; 
    - X17 = amount of bill statement in April, 2005. 
- X18-X23: Amount of previous payment (NT dollar). 
    - X18 = amount paid in September, 2005; 
    - X19 = amount paid in August, 2005; . . .;
    - etc...
    - X23 = amount paid in April, 2005. 




You will fit three different models (KNN, Logistic Regression, and Decision Tree Classifier) to predict credit card defaults and use gridsearch to find the best hyperparameters for those models. Then you will compare the performance of those three models on a test set to find the best one.  


## Process/Expectations

- You will be working in pairs for this assessment

### Please have ONE notebook and be prepared to explain how you worked in your pair.

1. Clean up your data set so that you can perform an EDA. 
    - This includes handling null values, categorical variables, removing unimportant columns, and removing outliers.
2. Perform EDA to identify opportunities to create new features.
    - [Great Example of EDA for classification](https://www.kaggle.com/stephaniestallworth/titanic-eda-classification-end-to-end) 
    - [Using Pairplots with Classification](https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166)
3. Engineer new features. 
    - Create polynomial and/or interaction features. 
    - Additionaly, you must also create **at least 2 new features** that are not interactions or polynomial transformations. 
        - *For example, you can create a new dummy variable that based on the value of a continuous variable (billamount6 >2000) or take the average of some past amounts.*
4. Perform some feature selection. 
    
5. You must fit **three** models to your data and tune **at least 1 hyperparameter** per model. 
6. Using the F-1 Score, evaluate how well your models perform and identify your best model.
7. Using information from your EDA process and your model(s) output provide insight as to which borrowers are more likely to deafult


In [1]:
# import libraries

import pandas as pd
import numpy as np


## 1. Data Cleaning

In [2]:
df = pd.read_csv('training_data.csv', index_col=0)

In [3]:
df.Y.value_counts()

0                             17471
1                              5028
default payment next month        1
Name: Y, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# Split data to be used in the models
# Create matrix of features
X = df.drop('Y', axis = 1) # grabs everything else but 'Survived'


# Create target variable
y = df['Y'] # y is the column we're trying to predict

In [6]:
df.index.sort_values()

Index(['1', '10', '100', '1000', '10001', '10003', '10004', '10005', '10007',
       '10009',
       ...
       '9982', '9985', '9986', '9988', '9990', '9992', '9995', '9997', '9999',
       'ID'],
      dtype='object', length=22500)

In [7]:
df.columns = df.loc['ID']

In [8]:
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object', name='ID')

In [9]:
df= df[df['SEX'] != 'SEX']

In [10]:
gender_opts= [1, 2]
Education_opts= [1 , 2, 3, 4]
Marital_opts= [1,2,3]
Pay_opts=[-1,1,2,3,4,5,6,7,8,9]

In [31]:
for column in df.columns:
    print (df[column].value_counts())

50000      2534
20000      1488
30000      1223
80000      1210
200000     1134
           ... 
327680        1
1000000       1
690000        1
740000        1
800000        1
Name: LIMIT_BAL, Length: 80, dtype: int64
2    13572
1     8927
Name: SEX, dtype: int64
2    10516
1     7919
3     3713
4      351
Name: EDUCATION, dtype: int64
2    12026
1    10195
3      278
Name: MARRIAGE, dtype: int64
29    1243
27    1100
28    1064
30    1026
26     959
25     904
31     902
32     878
33     874
34     859
24     851
35     841
36     812
37     799
38     720
39     703
23     690
40     656
41     613
42     597
44     512
43     495
45     462
46     421
22     420
47     340
48     339
49     326
50     298
51     269
53     241
52     236
54     184
55     154
56     147
58      89
57      89
59      62
60      56
21      56
61      45
62      32
64      24
63      24
66      21
65      17
67      13
69      10
70      10
68       5
72       3
73       3
75       2
71       2
74    

In [29]:
df['EDUCATION']= df['EDUCATION'].map(lambda x: int(x))
df['MARRIAGE']= df['MARRIAGE'].map(lambda x: int(x))

In [26]:
df['EDUCATION']= np.where(df['EDUCATION'] > 4, 4, df['EDUCATION'])
df['EDUCATION']= np.where(df['EDUCATION'] < 1, 4, df['EDUCATION'])

In [30]:
df['MARRIAGE']= np.where(df['MARRIAGE'] > 3, 3, df['MARRIAGE'])
df['MARRIAGE']= np.where(df['MARRIAGE'] < 1, 3, df['MARRIAGE'])

In [34]:
len(df['PAY_0'])

22499

In [36]:
np.where(df['BILL_AMT1'] == df['PAY_AMT1'], True, False).sum()

1602

In [38]:
len(df['PAY_0'])-17664

4835

## 2. EDA

## 3. Feature Engineering

## 4. Feature Selection

## 5. Model Fitting and Hyperparameter Tuning
KNN, Logistic Regression, Decision Tree

## 6. Model Evaluation

## 7. Final Model